In [1]:
import pandas as pd
import requests
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import missingno as msno

# **Analisis Exploratorio de la Presa Abelardo L. Rodriguéz**

En esta seccion se analizara los datos de acumuluacion de agua hm3 y datos metereologicos.

### Descripción de Variables Meteorológicas para el Análisis Hídrico

| Variable Meteorológica | Descripción de la Variable | Relación con el Análisis del Rol Hídrico |
| :--- | :--- | :--- |
| **Suma de Precipitación** | Cantidad total de lluvia acumulada en un día (expresada en mm). | **Entrada Directa:** Es la principal fuente de recarga de agua para el embalse. Permite cuantificar el volumen de agua que ingresa al sistema en cada evento. |
| **Evapotranspiración de Referencia (ET₀)** | Tasa combinada de evaporación desde la superficie del agua y transpiración de la vegetación, estimada bajo condiciones ideales. | **Salida Principal:** Representa la mayor pérdida de agua del embalse. Es crucial para calcular el balance hídrico y entender cuánta agua se pierde a la atmósfera. |
| **Humedad del Suelo (Soil Moisture)** | Cantidad de agua contenida en las diferentes capas del suelo (expresada como m³/m³). | **Regulador de Escorrentía:** Un suelo seco absorberá más lluvia, reduciendo el agua que llega a la presa. Un suelo saturado generará mayor escorrentía. Es clave para probar la hipótesis de recarga del acuífero. |
| **Temperatura Media** | Promedio de la temperatura del aire a lo largo del día (en °C). | **Catalizador de Evaporación:** Acelera la evaporación. Temperaturas más altas aumentan la energía disponible para que el agua cambie de estado, incrementando las pérdidas. |
| **Déficit de Presión de Vapor (VPD)** | Mide la "sed" de la atmósfera, indicando la diferencia entre la humedad actual y la máxima que podría contener el aire. | **Motor de Evaporación:** Es un indicador más directo y potente que la temperatura para predecir la tasa de evaporación. Un VPD alto significa una pérdida de agua muy rápida. |
| **Velocidad del Viento** | Velocidad media o máxima del viento (en km/h o m/s). | **Acelerador de Evaporación:** El viento remueve la capa de aire húmedo sobre la superficie del agua, facilitando que más agua se evapore. Acelera las pérdidas de agua. |
| **Radiación de Onda Corta** | Cantidad total de energía solar que llega a la superficie. | **Fuente de Energía Primaria:** Es la energía que calienta el agua y el suelo, siendo el combustible principal para el proceso de evaporación. |
| **Humedad Relativa** | Porcentaje de saturación de vapor de agua en el aire. | **Freno de Evaporación:** Un aire más húmedo (humedad relativa alta) tiene menos capacidad para absorber más vapor de agua, ralentizando la tasa de evaporación. |
| **Cobertura de Nubes** | Porcentaje del cielo cubierto por nubes. | **Regulador de Energía:** Las nubes bloquean la radiación solar, reduciendo la energía disponible para la evaporación. Los días nublados tienen menos pérdidas de agua. |

Uso de Suelo y Vegetacion 2021
http://www.conabio.gob.mx/informacion/gis/maps/geo/usv250s7gw.zip



Ahora contamos con un dataframe con todos los datos de la capacidad de la presa de 1941 - 1959 y 1970 - actualidad.

Seguimos con los datos de evapotranspiración, para esto usaremos Open-Meteo en su API historica.

# Descarga de datos

## Descarga de datos de presa

En esta seccion se realiza la descarga de datos, la cual termina en la conversion del dataframe en un archivo para no tener que estar descargando repetidamente la informacion.


En la pagina de datos abiertos del estado de sonora encontramos 8 (El de 1960-1969 falta) datasets con informacion de la capacidad de cada presa, estos son los links:

In [2]:
links_cap = [
    "https://datos.sonora.gob.mx/dataset/ee8f639f-5e89-46ae-93b9-934b05fc6233/resource/f2333c5e-26ea-47c8-93ee-fda5e9106672/download/hidrico_sonora_1941-1949.xlsx",
    "https://datos.sonora.gob.mx/dataset/ee8f639f-5e89-46ae-93b9-934b05fc6233/resource/8826c8a0-75f2-49b0-bf48-907b57c3495d/download/hidrico_sonora_1950-1959.xlsx",
    "https://datos.sonora.gob.mx/dataset/ee8f639f-5e89-46ae-93b9-934b05fc6233/resource/c05fe66a-bdeb-4d53-ba93-beb315b98901/download/hidrico_sonora_1970-1979.xlsx",
    "https://datos.sonora.gob.mx/dataset/ee8f639f-5e89-46ae-93b9-934b05fc6233/resource/ffe3c845-1fc7-4a6c-ab21-3a1db1740fdd/download/hidrico_sonora_1980-1989.xlsx",
    "https://datos.sonora.gob.mx/dataset/ee8f639f-5e89-46ae-93b9-934b05fc6233/resource/fabf0415-7c55-4b01-948f-5cedfb58d4b3/download/hidrico_sonora_1990-1999.xlsx",
    "https://datos.sonora.gob.mx/dataset/ee8f639f-5e89-46ae-93b9-934b05fc6233/resource/458d5daa-d0d1-452d-9524-5ef871f7cbe9/download/hidrico_sonora_2000-2009.xlsx",
    "https://datos.sonora.gob.mx/dataset/ee8f639f-5e89-46ae-93b9-934b05fc6233/resource/43780905-3659-40c2-808d-471775de7664/download/hidrico_sonora_2010-2019.xlsx",
    "https://datos.sonora.gob.mx/dataset/ee8f639f-5e89-46ae-93b9-934b05fc6233/resource/213a4f46-7cd1-412f-aeec-4fab8193f51f/download/hidrico_sonora_2020-actualidad2024.xlsx",
]

In [3]:
# Creamos archivos CSV para cada uno y los agregamos a un array que guarde los paths
paths = []
for link in links_cap:
    # Extraer el nombre del archivo del link y cambiar la extensión a .csv
    file_name = link.split("/")[-1].replace(".xlsx", ".csv")
    
    # Leer el archivo Excel desde el link y guardarlo como CSV
    try:
        df = pd.read_excel(link)
        df.to_csv(file_name, index=False)
        paths.append(file_name)
        print(f"Archivo creado: {file_name}")
    except Exception as e:
        print(f"No se pudo procesar el link {link}: {e}")

Archivo creado: hidrico_sonora_1941-1949.csv
Archivo creado: hidrico_sonora_1950-1959.csv
Archivo creado: hidrico_sonora_1970-1979.csv
Archivo creado: hidrico_sonora_1980-1989.csv
Archivo creado: hidrico_sonora_1990-1999.csv
Archivo creado: hidrico_sonora_2000-2009.csv
Archivo creado: hidrico_sonora_2010-2019.csv
Archivo creado: hidrico_sonora_2020-actualidad2024.csv


In [4]:
# Diccionario de claves de presas en Sonora
presas_sonora = {
    "LCDSO": "Presa Lázaro Cárdenas (La Angostura)",
    "CHTSO": "Presa Cuauhtémoc",
    "ARLSO": "Presa Abelardo L. Rodríguez",
    "AOBSO": "Presa Álvaro Obregón (Oviáchic)",
    "ARCSO": "Presa Adolfo Ruiz Cortines (Mocúzari)",
    "PECSO": "Presa Plutarco Elías Calles (El Novillo)",
    "AGZCH": "Presa Abraham González",
    "PMOSO": "Presa Ing. Rodolfo Félix Valdés (El Molinito)",
    "IRASO": "Presa Ignacio R. Alatorre",
    "BICSO": "Presa Bicentenario"
}

In [5]:
# Ejemplo de uso:
print(presas_sonora["AOBSO"])  # Devuelve: Presa Álvaro Obregón (Oviáchic)

Presa Álvaro Obregón (Oviáchic)


In [8]:
#Hacemos una prueba para ver como manejar correctamente la data
df = pd.read_csv("D:/Kevin/MCD/arhbpalr/arhbpalr/data/raw/hidrico_sonora_1941-1949.csv", names=["clave", "fecha", "almacenamiento_hm3"], skiprows=1, header=None)
df.head()

,clave,fecha,almacenamiento_hm3
0,LCDSO,1941/07/01,0.13
1,CHTSO,1941/07/01,-
2,CHTSO,1941/07/02,-
3,LCDSO,1941/07/02,0.15
4,LCDSO,1941/07/03,0.16


In [9]:
# Ahora hacemos una tabla con todos los datos
df_cap = pd.DataFrame()
for path in paths:
  df = pd.read_csv(path, names=["clave", "fecha", "almacenamiento_hm3"], skiprows=1, header=None)
  df_cap = pd.concat([df_cap, df])
  print("Archivo agregado: ", path)
print(df_cap.shape)
df_cap.head()

Archivo agregado:  hidrico_sonora_1941-1949.csv
Archivo agregado:  hidrico_sonora_1950-1959.csv
Archivo agregado:  hidrico_sonora_1970-1979.csv
Archivo agregado:  hidrico_sonora_1980-1989.csv
Archivo agregado:  hidrico_sonora_1990-1999.csv
Archivo agregado:  hidrico_sonora_2000-2009.csv
Archivo agregado:  hidrico_sonora_2010-2019.csv
Archivo agregado:  hidrico_sonora_2020-actualidad2024.csv
(202993, 3)


,clave,fecha,almacenamiento_hm3
0,LCDSO,1941/07/01,0.13
1,CHTSO,1941/07/01,-
2,CHTSO,1941/07/02,-
3,LCDSO,1941/07/02,0.15
4,LCDSO,1941/07/03,0.16


In [10]:
print("Valores únicos en la columna 'clave':")
print(df_cap['clave'].unique())

Valores únicos en la columna 'clave':
['LCDSO' 'CHTSO' 'ARLSO' 'AOBSO' 'ARCSO' 'PECSO' 'AGZCH' 'PMOSO' 'IRASO'
 'BICSO']


In [11]:
# De estos datos vamos a seleccionar solo los de la Presa Abelardo
df_arlso = df_cap[df_cap["clave"] == "ARLSO"].copy() # Use .copy() to avoid SettingWithCopyWarning
df_arlso.loc[:, "almacenamiento_hm3"] = df_arlso["almacenamiento_hm3"].astype(float)
df_arlso.loc[:, "fecha"] = pd.to_datetime(df_arlso["fecha"], format="mixed") # Corrected date format
print(df_arlso.shape)
display(df_arlso.head(20))

(24631, 3)


,clave,fecha,almacenamiento_hm3
4228,ARLSO,1947-04-14 00:00:00,0.09
4229,ARLSO,1947-04-15 00:00:00,0.11
4234,ARLSO,1947-04-16 00:00:00,0.11
4235,ARLSO,1947-04-17 00:00:00,0.12
4240,ARLSO,1947-04-18 00:00:00,0.12
4241,ARLSO,1947-04-19 00:00:00,0.13
4246,ARLSO,1947-04-20 00:00:00,0.14
4247,ARLSO,1947-04-21 00:00:00,0.14
4252,ARLSO,1947-04-22 00:00:00,0.15
4253,ARLSO,1947-04-23 00:00:00,0.15


In [21]:
# Guardar como CSV
df_arlso.to_csv('df_arlso.csv', index=False)
print("Archivo 'df_arlso.csv' guardado exitosamente.")

# Guardar como JSON
df_arlso.to_json('df_arlso.json', orient='records', date_format='iso')
print("Archivo 'df_arlso.json' guardado exitosamente.")

# Guardar como Parquet
# Convertir el DataFrame a una tabla de PyArrow
table_arlso = pa.Table.from_pandas(df_arlso)
# Escribir la tabla a un archivo Parquet
pq.write_table(table_arlso, 'df_arlso.parquet')
print("Archivo 'df_arlso.parquet' guardado exitosamente.")

Archivo 'df_arlso.csv' guardado exitosamente.
Archivo 'df_arlso.json' guardado exitosamente.
Archivo 'df_arlso.parquet' guardado exitosamente.


## **API OPENMETEO**   

*Aqui se deja el codigo para cargar datos metereologicos mas completos de openmeteo*

Ya no es necesario volver a cargar las celdas una vez que se ejecutan por primera vez

In [12]:
API_URL_full = "https://archive-api.open-meteo.com/v1/archive?latitude=29.1026&longitude=-110.9773&start_date=1940-01-01&end_date=2024-12-31&daily=precipitation_sum,rain_sum,et0_fao_evapotranspiration,temperature_2m_mean,temperature_2m_max,precipitation_hours,wind_speed_10m_max,wind_speed_10m_mean,vapour_pressure_deficit_max,shortwave_radiation_sum,relative_humidity_2m_mean,cloud_cover_mean,soil_moisture_0_to_100cm_mean,soil_moisture_0_to_7cm_mean,soil_moisture_28_to_100cm_mean,soil_moisture_7_to_28cm_mean&timezone=auto"

In [13]:
response = requests.get(API_URL_full)
if response.status_code == 200:
    data_full = response.json()
    print(data_full)
else:
    print('Error:', response.status_code)

{'latitude': 29.06854, 'longitude': -110.975876, 'generationtime_ms': 376.66118144989014, 'utc_offset_seconds': -25200, 'timezone': 'America/Hermosillo', 'timezone_abbreviation': 'GMT-7', 'elevation': 214.0, 'daily_units': {'time': 'iso8601', 'precipitation_sum': 'mm', 'rain_sum': 'mm', 'et0_fao_evapotranspiration': 'mm', 'temperature_2m_mean': '°C', 'temperature_2m_max': '°C', 'precipitation_hours': 'h', 'wind_speed_10m_max': 'km/h', 'wind_speed_10m_mean': 'km/h', 'vapour_pressure_deficit_max': 'kPa', 'shortwave_radiation_sum': 'MJ/m²', 'relative_humidity_2m_mean': '%', 'cloud_cover_mean': '%', 'soil_moisture_0_to_100cm_mean': 'm³/m³', 'soil_moisture_0_to_7cm_mean': 'm³/m³', 'soil_moisture_28_to_100cm_mean': 'm³/m³', 'soil_moisture_7_to_28cm_mean': 'm³/m³'}, 'daily': {'time': ['1940-01-01', '1940-01-02', '1940-01-03', '1940-01-04', '1940-01-05', '1940-01-06', '1940-01-07', '1940-01-08', '1940-01-09', '1940-01-10', '1940-01-11', '1940-01-12', '1940-01-13', '1940-01-14', '1940-01-15', '

In [15]:
df_meteo = pd.DataFrame(data_full['daily'])

In [16]:
df_meteo = pd.DataFrame(data_full['daily'])
df_meteo["time"] = pd.to_datetime(df_meteo["time"])
print(df_meteo.shape)
df_meteo.head()

(31047, 17)


,time,precipitation_sum,rain_sum,et0_fao_evapotranspiration,temperature_2m_mean,temperature_2m_max,precipitation_hours,wind_speed_10m_max,wind_speed_10m_mean,vapour_pressure_deficit_max,shortwave_radiation_sum,relative_humidity_2m_mean,cloud_cover_mean,soil_moisture_0_to_100cm_mean,soil_moisture_0_to_7cm_mean,soil_moisture_28_to_100cm_mean,soil_moisture_7_to_28cm_mean
0,1940-01-01,0.0,0.0,3.19,15.5,23.7,0.0,11.9,6.8,2.33,14.54,42,41,0.053,0.053,0.051,0.062
1,1940-01-02,0.0,0.0,2.86,15.5,23.3,0.0,13.0,7.2,2.30,11.71,42,77,0.053,0.053,0.051,0.061
2,1940-01-03,0.0,0.0,3.68,17.0,25.5,0.0,18.2,9.7,2.60,14.10,42,63,0.053,0.052,0.051,0.061
3,1940-01-04,0.0,0.0,2.75,16.1,22.8,0.0,13.2,6.5,2.00,11.94,52,68,0.053,0.052,0.051,0.061
4,1940-01-05,0.1,0.1,2.19,15.1,21.0,1.0,19.8,11.3,1.38,8.83,66,78,0.053,0.052,0.051,0.061


In [17]:
pip install pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Codigo para guardar los archivos que se generan con la API de openmeteo

Esto funciona para no estar llamando la API de openmeteo y que se esten acabando las llamadas que son gratis (10mil aprox)

In [18]:
import pyarrow as pa
import pyarrow.parquet as pq

# Convert the DataFrame to a PyArrow Table
table = pa.Table.from_pandas(df_meteo)

# Write the Table to a Parquet file
pq.write_table(table, 'meteo_data_full.parquet')

In [19]:
# Guardar como CSV
df_meteo.to_csv('meteo_data_full.csv', index=False)
print("Archivo 'meteo_data_full.csv' guardado exitosamente.")

Archivo 'meteo_data_full.csv' guardado exitosamente.


In [20]:
# Guardar como JSON
df_meteo.to_json('meteo_data_full.json', orient='records', date_format='iso')
print("Archivo 'meteo_data_full.json' guardado exitosamente.")

Archivo 'meteo_data_full.json' guardado exitosamente.
